<a href="https://colab.research.google.com/github/sarnavadatta/Predictive-Modelling/blob/main/LoanDefault_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls '/content/drive/My Drive/DS projects/Loan Default Prediction/'

data_loan_default.xlsx	LoanDefault_ML.ipynb


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('/content/drive/My Drive/DS projects/Loan Default Prediction/data_loan_default.xlsx')
df.head()

,loan_amnt,term,installment,emp_length,home_ownership,verification_status,issue_d,purpose,dti,earliest_cr_line,...,pub_rec,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies,log_annual_inc,fico_score,log_revol_bal,charged_off
0,5675,36,189.56,2.0,RENT,Source Verified,2014-12-01,car,15.55,1992,...,0,15.0,15,Individual,0.0,0.0,4.698979,697.0,3.464936,0
1,7000,36,242.63,1.0,RENT,Source Verified,2014-12-01,car,6.78,2002,...,0,69.2,5,Individual,0.0,0.0,4.301052,717.0,3.431364,0
2,3500,36,116.24,10.0,MORTGAGE,Not Verified,2014-12-01,car,10.65,1997,...,0,12.6,35,Individual,2.0,0.0,4.681250,742.0,3.765520,0
3,10000,36,316.47,10.0,MORTGAGE,Not Verified,2014-12-01,car,21.74,1996,...,0,23.7,22,Individual,4.0,0.0,4.875067,707.0,3.424555,0
4,1500,36,46.31,2.0,OWN,Not Verified,2014-12-01,car,12.17,1997,...,0,46.5,13,Individual,2.0,0.0,4.812920,712.0,4.448211,0


**Exploratory Data Analysis**

In [ ]:
print(df.isnull().sum())

loan_amnt                  0
term                       0
installment                0
emp_length               425
home_ownership             0
verification_status        0
issue_d                    0
purpose                    0
dti                        0
earliest_cr_line           0
open_acc                   0
pub_rec                    0
revol_util                14
total_acc                  0
application_type           0
mort_acc                1756
pub_rec_bankruptcies      17
log_annual_inc             0
fico_score                 0
log_revol_bal              0
charged_off                0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8929 entries, 0 to 8928
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   loan_amnt             8929 non-null   int64         
 1   term                  8929 non-null   int64         
 2   installment           8929 non-null   float64       
 3   emp_length            8504 non-null   float64       
 4   home_ownership        8929 non-null   object        
 5   verification_status   8929 non-null   object        
 6   issue_d               8929 non-null   datetime64[ns]
 7   purpose               8929 non-null   object        
 8   dti                   8929 non-null   float64       
 9   earliest_cr_line      8929 non-null   int64         
 10  open_acc              8929 non-null   int64         
 11  pub_rec               8929 non-null   int64         
 12  revol_util            8915 non-null   float64       
 13  total_acc         

In [ ]:
# Calculate unique values for object type columns
for col in df.select_dtypes(include=['object']).columns:
    print(f"Unique values in '{col}': {df[col].unique()}")
    print(f"Number of unique values in '{col}': {df[col].nunique()}")
    print("-" * 20)


Unique values in 'home_ownership': ['RENT' 'MORTGAGE' 'OWN']
Number of unique values in 'home_ownership': 3
--------------------
Unique values in 'verification_status': ['Source Verified' 'Not Verified' 'Verified']
Number of unique values in 'verification_status': 3
--------------------
Unique values in 'purpose': ['car']
Number of unique values in 'purpose': 1
--------------------
Unique values in 'application_type': ['Individual' 'Joint App']
Number of unique values in 'application_type': 2
--------------------


In [ ]:
# verification_status status column has three value: ['Source Verified' 'Not Verified' 'Verified']. We replace 'Source Verified' to 'Verified'.
df['verification_status'] = df['verification_status'].replace({'Source Verified': 'Verified'})
df['verification_status'].unique()

array(['Verified', 'Not Verified'], dtype=object)

In [ ]:
# purpose column only have one variable. Thus completely drop the column since it will not affect the prediction
df = df.drop(['purpose'], axis=1)

In [ ]:
print(df['application_type'].value_counts())

application_type
Individual    8908
Joint App       21
Name: count, dtype: int64


In [ ]:
# since almost all the application type is Individual, we are going to drop this column as well.
df = df.drop(['application_type'], axis=1)

In [ ]:
# emp_length and revol_util contain continuous numeric values. For those features, we replace the missing values by their respective median.
# For pub_rec_bankruptcies, we replace it with mode (most occured value, which is 0 in this scenario).

df['emp_length'] = df['emp_length'].fillna(df['emp_length'].median())
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].median())
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0)

In [ ]:
# use knn imputer for  the missing values of mort_acc

from sklearn.impute import KNNImputer

# Create a KNN imputer object
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Select the columns with missing values you want to impute
cols_to_impute = ['mort_acc']

# Fit and transform the imputer on the selected columns
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])


In [ ]:
print(df.isnull().sum())

loan_amnt               0
term                    0
installment             0
emp_length              0
home_ownership          0
verification_status     0
issue_d                 0
dti                     0
earliest_cr_line        0
open_acc                0
pub_rec                 0
revol_util              0
total_acc               0
mort_acc                0
pub_rec_bankruptcies    0
log_annual_inc          0
fico_score              0
log_revol_bal           0
charged_off             0
dtype: int64


In [ ]:
# We notice that the earliest_cr_line variable provides the date of the earliest credit line of the customer.
# However, this does not provide any useful information because the loans are issued at different dates.
# Therefore, we transform this variable to obtain the number of years between the earliest credit line and the loan issue date.
# We also remove the issue_d feature.

df['yr_since_earliest_cr_line'] = pd.DatetimeIndex(df['issue_d']).year - df['earliest_cr_line']
df = df.drop(['earliest_cr_line','issue_d'], axis=1)


In [ ]:
df.head()

,loan_amnt,term,installment,emp_length,home_ownership,verification_status,dti,open_acc,pub_rec,revol_util,total_acc,mort_acc,pub_rec_bankruptcies,log_annual_inc,fico_score,log_revol_bal,charged_off,yr_since_earliest_cr_line
0,5675,36,189.56,2.0,RENT,Verified,15.55,7,0,15.0,15,0.0,0.0,4.698979,697.0,3.464936,0,22
1,7000,36,242.63,1.0,RENT,Verified,6.78,3,0,69.2,5,0.0,0.0,4.301052,717.0,3.431364,0,12
2,3500,36,116.24,10.0,MORTGAGE,Not Verified,10.65,16,0,12.6,35,2.0,0.0,4.681250,742.0,3.765520,0,17
3,10000,36,316.47,10.0,MORTGAGE,Not Verified,21.74,7,0,23.7,22,4.0,0.0,4.875067,707.0,3.424555,0,18
4,1500,36,46.31,2.0,OWN,Not Verified,12.17,8,0,46.5,13,2.0,0.0,4.812920,712.0,4.448211,0,17


**Train-Test Split & Model Training**

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(['charged_off'], axis=1)
y = df['charged_off']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y.value_counts()

,count
charged_off,
0,7608
1,1321
